In [68]:
import pandas as pd
import re
import time
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime

def setup_driver():
    service = Service(executable_path="chromedriver.exe")
    driver = webdriver.Chrome(service=service)
    return driver

def accept_cookies(driver):
    try:
        wait = WebDriverWait(driver, 20)
        element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'amgdprcookie-button')))
        buttons = driver.find_elements(By.CLASS_NAME, 'amgdprcookie-button')
        for btn in buttons:
            if '-settings' in btn.get_attribute('class') and '-save' in btn.get_attribute('class'):
                btn.click()
                break
        else:
            print("Cookies button not found.")
    except Exception as e:
        print(f"Error finding element: {e}")

def login(driver, username, password):
    try:
        wait = WebDriverWait(driver, 20)
        username_field = wait.until(EC.visibility_of_element_located((By.ID, 'gigya-loginID-20503411558167264')))
        password_field = wait.until(EC.visibility_of_element_located((By.ID, 'gigya-password-102120096138430450')))
        username_field.send_keys(username)
        password_field.send_keys(password)
        password_field.send_keys(Keys.RETURN)
    except Exception as e:
        print(f"Error finding element: {e}")

def get_order_ids(driver):
    order_ids = []
    try:
        wait = WebDriverWait(driver, 20)
        wait.until(EC.visibility_of_element_located((By.ID, 'my-orders-table')))
        orders = driver.find_elements(By.CSS_SELECTOR, '#my-orders-table tbody tr')
        for order in orders:
            order_id_element = order.find_element(By.CSS_SELECTOR, 'td.col.id span')
            order_ids.append(order_id_element.text)
    except Exception as e:
        print(f"Error finding order IDs: {e}")
    return order_ids

def find_and_click_order(driver, target_order_id):
    try:
        wait = WebDriverWait(driver, 20)
        wait.until(EC.visibility_of_element_located((By.ID, 'my-orders-table')))
        orders = driver.find_elements(By.CSS_SELECTOR, '#my-orders-table tbody tr')
        for order in orders:
            order_id_element = order.find_element(By.CSS_SELECTOR, 'td.col.id span')
            if order_id_element.text == target_order_id:
                view_link = order.find_element(By.CSS_SELECTOR, 'a.action.view')
                view_link.click()
                break
    except Exception as e:
        print(f"Error finding or clicking order: {e}")

def extract_volume(input_string):
    volume_match = re.search(r"Vol\. (\d+)", input_string)
    return volume_match.group(1) if volume_match else None

def get_order_info_value(order_info, search_string):
    text = order_info.text
    value_match = re.search(f"{search_string}: (.+)", text)
    return value_match.group(1)

def convert_date_pt_to_en(date_str):
    months_pt_to_en = {
        "janeiro": "January",
        "fevereiro": "February",
        "março": "March",
        "abril": "April",
        "maio": "May",
        "junho": "June",
        "julho": "July",
        "agosto": "August",
        "setembro": "September",
        "outubro": "October",
        "novembro": "November",
        "dezembro": "December"
    }
    for pt_month, en_month in months_pt_to_en.items():
        if pt_month in date_str:
            date_str = date_str.replace(pt_month, en_month)
            break
    return datetime.strptime(date_str, "%d de %B de %Y")

def extract_order_data(driver, target_order_id):
    product_list = []
    wait = WebDriverWait(driver, 20)
    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'table-wrapper.order-items')))
    products = driver.find_elements(By.CLASS_NAME, 'product')
    order_info_element = driver.find_element(By.CLASS_NAME, 'order-title__info')
    
    for product in products:
        order_id = target_order_id
        title = product.find_element(By.CLASS_NAME, 'name').text
        quantity = product.find_element(By.CLASS_NAME, 'product-buy__qty').text.split(' ')[1]
        price = product.find_element(By.CLASS_NAME, 'price').text
        volume = extract_volume(title)
        status_text = get_order_info_value(order_info_element, 'Status')
        order_date = get_order_info_value(order_info_element, 'Data do pedido')
        order_date_obj = convert_date_pt_to_en(order_date)
        order_value = get_order_info_value(order_info_element, 'Total')
        order_value_obj = re.search(r'R\$\d+,\d{2}', order_value).group(0)
        
        product_list.append({
            'Order_id': order_id, 
            'Title': title, 
            'Quantity': quantity, 
            'Price': price, 
            'Volume': volume, 
            'Status': status_text, 
            'Order Date': order_date_obj,
            'Order Value': order_value_obj
        })
    
    return pd.DataFrame(product_list)

def main():
    driver = setup_driver()
    url = 'https://panini.com.br/sales/order/history/'
    driver.get(url)
    
    username = os.getenv('USERNAME')
    password = os.getenv('PASSWORD')
    
    accept_cookies(driver)
    login(driver, username, password)
    
    time.sleep(10)  # Wait for redirection after login
    
    order_ids = get_order_ids(driver)
    
    all_dataframes = []
    for order_id in order_ids:
        find_and_click_order(driver, order_id)
        df = extract_order_data(driver, order_id)
        all_dataframes.append(df)
        driver.back()  # Go back to the orders list page
    
    final_df = pd.concat(all_dataframes, ignore_index=True)
    
    print(final_df)
    final_df.to_csv('orders.csv', index=False)
    
    driver.quit()

if __name__ == "__main__":
    main()


     Order_id                                              Title Quantity  \
0   007262539  Dragon Ball Vol. 32 - Edição Definitiva (Capa ...        1   
1   007262539  Dragon Ball Vol. 33 - Edição Definitiva (Capa ...        1   
2   007262539  Dragon Ball Vol. 31 - Edição Definitiva (Capa ...        1   
3   007262539  Dragon Ball Vol. 28 - Edição Definitiva (Capa ...        1   
4   007262539  Dragon Ball Vol. 26 - Edição Definitiva (Capa ...        1   
5   007262539  Dragon Ball Vol. 30 - Edição Definitiva (Capa ...        1   
6   007262539  Dragon Ball Vol. 29 - Edição Definitiva (Capa ...        1   
7   007257601  Assinatura Dragon Ball Edição Definitiva - Pla...        1   
8   007257598  Dragon Ball Vol. 1 - Edição Definitiva (Capa D...        1   
9   007257598  Dragon Ball Vol. 2 - Edição Definitiva (Capa D...        1   
10  007257598  Dragon Ball Vol. 3 - Edição Definitiva (Capa D...        1   
11  007257598  Dragon Ball Vol. 4 - Edição Definitiva (Capa D...        1   